<a href="https://colab.research.google.com/github/MMRES-PyBootcamp/MMRES-python-bootcamp2021/blob/master/Exercise_Prostate_Cancer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Session 5 - Data visualization
> TODO An introduction on Pandas intermediate level concepts. Here we will present how to *manipulate* the data stored in a Pandas DataFrame, no matter if their Pandas Series store numerical, text or more complex data types. Finally we will introduce you some tools to *reshape* and/or *aggregate* data.

## Outline TODO
 * [DataFrame transformations](#DataFrame-transformations)
   * [DataFrame numerical transformations](#DataFrame-numerical-transformations)
   * [DataFrame text transformations](#DataFrame-text-transformations)
   * [Arbitrary transformations using `.apply()` method](#Arbitrary-transformations-using-.apply()-method) 
 * [Exporting DataFrames](#Exporting-DataFrames)
 * [Grouping-by and aggregating DataFrames](#Grouping-by-and-aggregating-DataFrames)
 * [Pivoting DataFrames](#Pivoting-DataFrames)
 * [Melting DataFrames](#Melting-DataFrames)

<div class="alert alert-block alert-success"><b>Practice:</b> Practice cells announce exercises that you should try during the current boot camp session.
</div>

<div class="alert alert-block alert-warning"><b>Extension:</b> Extension cells correspond to exercises (or links to contents) that are a bit more advanced. We recommend to try them after the current boot camp session.
</div>

<div class="alert alert-block alert-info"><b>Tip:</b> Tip cells just give some advice or complementary information.
</div>

<div class="alert alert-block alert-danger"><b>Caveat:</b> Caveat cells warn you about the most common pitfalls one founds when starts his/her path learning Python.

</div>

**This document is devised as a tool to enable your self-learning process. If you get stuck at some step or need any kind of help, please don't hesitate to raise your hand and ask for the teacher's guidance.**

---

## Introduction

In this session, we will explore the tools that Python offers to visualize data. The *art* of making nice plots is something that takes some time, but getting our first plots is really simple. In this tutorial, we will focus in the libraries [Matplotlib](http://matplotlib.org/) and [Seaborn](https://seaborn.pydata.org/). Matplotlib provides an absolute control on what you are plotting but often requires quite many code lines. With Seaborn you can get really nice plots in just a couple code lines (that's the reason why we choose this package). Like many other Python plotting packages, Seaborn is based in Matplotlib, and at the end of the day, we will leverage the best of each package to make our plots.

<div class="alert alert-block alert-info"><b>Tip:</b>

Each Python user has its own favourite plotting packages. In my case, despite I started with Seaborn, I recently switched to [Plotnine](https://plotnine.readthedocs.io/en/stable/) as my default. When I need plots with some degree of interactivity, I love using [Bokeh](https://docs.bokeh.org/en/latest/) instead. Try to find the packages that better fits your needs.    
</div>

We think that working with a true data set using a hands-on approach is the best way to learn the basics on data visualization with Matplotlib and Seaborn. For this reason we will try to reproduce some plots from the [Figure 4](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7960507/figure/Fig4/) appearing in a [Nature Medicine](https://www.nature.com/nm/) paper entitled [*Transcriptional mediators of treatment resistance in lethal prostate cancer*](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7960507/).

Let's start by importing the necessary packages. We will only need the `Seaborn` package and *class* called `pyplot` from the `Matplotlib` package (which has most of what we usually need for plotting):

In [ ]:
# Load packages with their corresponding alias
import pandas as pd

# Load plotting packages/classes with their corresponding alias
import seaborn as sns
import matplotlib.pyplot as plt

## Plotting categorical data

Let's import the metadata of the prostate cancer data set to show how easy is generate plots with Seaborn and Matplotlib:

In [ ]:
# Define the relative path towards the folder with our data files
path = 'datasets/prostate_cancer_data/'

# Reading file and storing it as a DataFrame
df_metadata = pd.read_csv(filepath_or_buffer=f'{path}scp_metadata.tsv', sep='\t', index_col=0, skiprows=[1])

Remember that it is always a good idea to get a bit familiar with the data you have between hands:

In [ ]:
# DataFrame general information
df_metadata.info()

In [ ]:
# DataFrame head (five first rows)
df_metadata.head()

In [ ]:
# DataFrame tail (last first rows)
df_metadata.tail()

It seems that some columns in `df_metadata` (`species`, `species__ontology_label`, `disease`, `disease__ontology_label`...) have the same value, let's check it out:

In [ ]:
# Get `df_metadata` (whole DataFrame) unique values
df_metadata.nunique()

In [ ]:
df_metadata['donor_id'] = df_metadata['donor_id'].apply(str)

# TODO
ax = sns.histplot(
    data=df_metadata,
    x="biosample_id",
    hue="donor_id",
    alpha=1
    # multiple="stack",
    # palette="light:m_r",
    # edgecolor=".3",
    # linewidth=.5,
    # log_scale=True,
)

# TODO
ax.tick_params(axis='x', rotation=90)

# TODO
sns.move_legend(ax, "upper left", bbox_to_anchor=(1, 1))

In [ ]:
df_metadata

In [ ]:
# Define the relative path towards the folder with our data files
path = 'datasets/prostate_cancer_data/'

# Reading files of interest and storing them as DataFrames
df_all = pd.read_csv(filepath_or_buffer=f'{path}scp_clustering.tsv', sep='\t', index_col=0, skiprows=[1])
# df_nkt = pd.read_csv(filepath_or_buffer=f'{path}scp_nk_t_clustering.tsv', sep='\t', index_col=0, skiprows=[1])
df_metadata = pd.read_csv(filepath_or_buffer=f'{path}scp_metadata.tsv', sep='\t', index_col=0, skiprows=[1])

# TODO
df_exp = pd.read_csv(filepath_or_buffer=f'{path}scp_tpm.tsv.gz', sep='\t', index_col=0).T
df_exp.index=df_exp.index.astype(int)

# TODO
df = df_exp.merge(df_all, right_index=True, left_index=True, how='inner')

In [ ]:
df

In [ ]:
# TODO
print(df_all.info())

Remember that is always recommended to get a bit familiar with the data 

In [ ]:
#imports
import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import zscore


## <strong>Data Import</strong>
The supplementary data of this publication is available at the Broad Institute’s Single Cell Portal (“[SCP](https://singlecell.broadinstitute.org/single_cell)”). For this bootcamp, I have uploaded the data in our GitHub repository so that you don't need to create an SCP account. However, I encourage you to explore this resource by your own because it contains lots of interesting data that you can use for your projects and the user interface is very intuitive and allows you to perform some exploratory visualizations.

<br>

Let's have a look at the entry for this project: [SCP1244](https://singlecell.broadinstitute.org/single_cell/study/SCP1244/transcriptional-mediators-of-treatment-resistance-in-lethal-prostate-cancer)




In [ ]:
# Reading an Excel SpreadSheet and storing it in as a DataFrame called `df`
df_all = pd.read_csv(filepath_or_buffer='datasets/prostate_cancer_data/scp_clustering.tsv',sep='\t',index_col=0,skiprows=[1])
df_nkt = pd.read_csv(filepath_or_buffer='datasets/prostate_cancer_data/scp_nk_t_clustering.tsv',sep='\t',index_col=0,skiprows=[1])
df_metadata = pd.read_csv(filepath_or_buffer='datasets/prostate_cancer_data/scp_metadata.tsv',sep='\t',index_col=0,skiprows=[1])

# Return the DataFrame
df_all.head()

In [ ]:
df_all.info()

In [ ]:
# import workshop data directly from a URL:
df_all = pd.read_csv('https://raw.githubusercontent.com/MMRES-PyBootcamp/MMRES-python-bootcamp2021/master/prostate_cancer_data/scp_clustering.tsv',sep='\t',index_col=0,skiprows=[1])
df_nkt = pd.read_csv('https://raw.githubusercontent.com/MMRES-PyBootcamp/MMRES-python-bootcamp2021/master/prostate_cancer_data/scp_nk_t_clustering.tsv',sep='\t',index_col=0,skiprows=[1])
df_metadata = pd.read_csv('https://raw.githubusercontent.com/MMRES-PyBootcamp/MMRES-python-bootcamp2021/master/prostate_cancer_data/scp_metadata.tsv',sep='\t',index_col=0,skiprows=[1])

In [ ]:
df_all

In [ ]:
# TODO
df_exp = pd.read_csv(filepath_or_buffer="datasets/prostate_cancer_data/scp_tpm.tsv.gz",
                     sep='\t',
                     index_col=0).T

# TODO
df_exp.index=df_exp.index.astype(int)

In [ ]:
df_exp

In [ ]:
# download, uncompress and import workshop data
os.getcwd()
if not os.path.exists('./assets/'):
  os.mkdir('./assets/')
!wget --quiet -O -  https://github.com/MMRES-PyBootcamp/MMRES-python-bootcamp2022/tree/main/datasets/prostate_cancer_data/scp_tpm.tsv.gz | gunzip > ./assets/scp_tpm.tsv
df_exp=pd.read_csv('./assets/scp_tpm.tsv',sep='\t',index_col=0).T
df_exp.index=df_exp.index.astype(np.int)

## <strong>Whole Dataset</strong>


In [ ]:
df=df_exp.merge(df_all,right_index=True,left_index=True,how='inner')


In [ ]:
df

In [ ]:

f, ax = plt.subplots(figsize=(6.5, 6.5))
sns.boxplot(data=df,x='cluster dominant cell type',y='CD3G')


In [ ]:
# let's order the cell types according to the expression of this marker gene
s_celltypes=sorted([(col,val['CD3G'].mean()) for col,val in df.groupby('cluster dominant cell type')], key=lambda x: x[1],reverse=True)
s_celltypes


In [ ]:

ordered_cells=np.array(s_celltypes).T[0]

f, ax = plt.subplots(figsize=(6.5, 6.5))
sns.boxplot(data=df,x='cluster dominant cell type',y='CD3G',order=ordered_cells)
ax.set_xticklabels(ordered_cells,rotation=90)

In [ ]:
f, ax = plt.subplots(figsize=(6.5, 6.5))

sns.scatterplot(x="X", y="Y",
                hue="cluster dominant cell type",
                hue_order=ordered_cells,
                linewidth=0,
                data=df, ax=ax)

ax.legend(bbox_to_anchor=(1.1, 1.05))

f, ax = plt.subplots(figsize=(6.5, 6.5))

sns.scatterplot(x="X", y="Y",
                hue="CD3G",
                palette=plt.cm.seismic,hue_norm=(0,500),hue_order="CD3G",
                linewidth=0,
                data=df, ax=ax)

ax.legend(bbox_to_anchor=(1.5, 1.05))

In [ ]:
sns.relplot(
    data=df, x="X", y="Y",
    col="cluster dominant cell type", hue="CD3G", 
    col_wrap=4, palette=plt.cm.seismic,hue_norm=(0,500), col_order=ordered_cells,
    kind="scatter"
)

## <strong>Cytotoxic lymphocyte populations</strong>


In [ ]:
df=df_exp.merge(df_nkt,right_index=True,left_index=True,how='inner')

In [ ]:
color_dict={
    'CD8+ PD1+ T cell':'#2a9a79', 
    'CD8+ CXCR4+ T cell':'#7eb459', 
    'CD8+ GNLY+ T cell':'#486aa5',
    'CD16+ NK cell':'#7e3689', 
    'CD4+ Treg cell':'#ec9034', 
    'CD4+ T cell':'#fd7971'
}

In [ ]:
# let's order the cell types according to the expression of CD3 marker gene
s_celltypes=sorted([(col,val['CD3G'].mean()) for col,val in df.groupby('cluster dominant cell type')], key=lambda x: x[1],reverse=True)
ordered_cells=np.array(s_celltypes).T[0]
f, ax = plt.subplots(figsize=(6.5, 6.5))
sns.boxplot(data=df,x='cluster dominant cell type',y='CD3G',order=ordered_cells, showfliers = False,palette=color_dict)
ax.set_xticklabels(ordered_cells,rotation=90)


In [ ]:
# let's order the cell types according to the expression of CD4 marker gene
s_celltypes=sorted([(col,val['CD4'].mean()) for col,val in df.groupby('cluster dominant cell type')], key=lambda x: x[1],reverse=True)
ordered_cells=np.array(s_celltypes).T[0]
f, ax = plt.subplots(figsize=(6.5, 6.5))
sns.boxplot(data=df,x='cluster dominant cell type',y='CD4',order=ordered_cells, showfliers = False,palette=color_dict)
ax.set_xticklabels(ordered_cells,rotation=90)


In [ ]:
# let's order the cell types according to the expression of CD16 (FCGR3B) marker gene
s_celltypes=sorted([(col,val['FCGR3B'].mean()) for col,val in df.groupby('cluster dominant cell type')], key=lambda x: x[1],reverse=True)
ordered_cells=np.array(s_celltypes).T[0]
f, ax = plt.subplots(figsize=(6.5, 6.5))
sns.boxplot(data=df,x='cluster dominant cell type',y='FCGR3B',order=ordered_cells, showfliers = False,palette=color_dict)
ax.set_xticklabels(ordered_cells,rotation=90)

In [ ]:
# let's order the cell types according to the expression of CD8 marker gene
s_celltypes=sorted([(col,val['CD8A'].mean()) for col,val in df.groupby('cluster dominant cell type')], key=lambda x: x[1],reverse=True)
ordered_cells=np.array(s_celltypes).T[0]
f, ax = plt.subplots(figsize=(6.5, 6.5))
sns.boxplot(data=df,x='cluster dominant cell type',y='CD8A',order=ordered_cells, showfliers = False,palette=color_dict)
ax.set_xticklabels(ordered_cells,rotation=90)


In [ ]:
ordered_cells=['CD16+ NK cell', 'CD8+ GNLY+ T cell', 'CD8+ CXCR4+ T cell','CD8+ PD1+ T cell']
f, ax = plt.subplots(figsize=(3, 6.5))
sns.stripplot(data=df,x='cluster dominant cell type',y='PDCD1',order=ordered_cells, palette=color_dict)
ax.set_xticklabels(ordered_cells,rotation=90)


In [ ]:
ordered_cells=['CD16+ NK cell', 'CD8+ GNLY+ T cell', 'CD8+ CXCR4+ T cell','CD8+ PD1+ T cell']
f, ax = plt.subplots(figsize=(3, 6.5))
sns.stripplot(data=df,x='cluster dominant cell type',y='CX3CR1',order=ordered_cells, palette=color_dict)
ax.set_xticklabels(ordered_cells,rotation=90)

## <strong>Heatmaps</strong>


In [ ]:
markers=['CD4','CD8A','CD3G','FCGR3B','PDCD1','HAVCR2','TOX','CX3CR1','CXCR4','GZMB']

In [ ]:
sns.heatmap(df[markers],cmap=plt.cm.seismic,vmin=0,vmax=500)

In [ ]:
clusters=df['cluster dominant cell type']
row_colors = clusters.map(color_dict)
sns.clustermap(df[markers].apply(lambda x:zscore(x),axis=0),cmap=plt.cm.seismic,vmin=-2,vmax=2,row_colors=row_colors.to_numpy())


In [ ]:
sns.clustermap(df[markers],z_score=1,cmap=plt.cm.seismic,vmin=-2,vmax=2,row_colors=row_colors.to_numpy())


In [ ]:
markers=['CD4','CD8A','FCGR3B','PDCD1','CXCR4','GNLY']
#markers=['CD4','CD8A','FCGR3B']
sns.clustermap(df[markers],z_score=1,cmap=plt.cm.seismic,vmin=-2,vmax=2,row_colors=row_colors.to_numpy())


## <strong> Statistics </strong>


https://medium.com/insights-school/learn-basic-statistics-with-python-cc0f45275929
